In [1]:
import os
import pandas as pd
import chardet

<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <p>Detection du format d'encodage du fichier source</p>
</div>

In [ ]:
with open('mon_fichier.csv', rb) as file :
    encodage = chardet.detect(file.read(10000))

print(encodage)

<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <p>Import du fichier source avec le bon encodage</p>
</div>

In [ ]:
pd.options.display.max_columns = None
df = pd.read_csv("raw_data/houses_Madrid.csv")
# df = pd.read_csv("raw_data/houses_Madrid.csv", encoding='utf_8_sig')

In [92]:
df.head(2)

,Unnamed: 0,id,title,subtitle,sq_mt_built,sq_mt_useful,n_rooms,n_bathrooms,n_floors,sq_mt_allotment,latitude,longitude,raw_address,is_exact_address_hidden,street_name,street_number,portal,floor,is_floor_under,door,neighborhood_id,operation,rent_price,rent_price_by_area,is_rent_price_known,buy_price,buy_price_by_area,is_buy_price_known,house_type_id,is_renewal_needed,is_new_development,built_year,has_central_heating,has_individual_heating,are_pets_allowed,has_ac,has_fitted_wardrobes,has_lift,is_exterior,has_garden,has_pool,has_terrace,has_balcony,has_storage_room,is_furnished,is_kitchen_equipped,is_accessible,has_green_zones,energy_certificate,has_parking,has_private_parking,has_public_parking,is_parking_included_in_price,parking_price,is_orientation_north,is_orientation_west,is_orientation_south,is_orientation_east
0,0,21742,"Piso en venta en calle de Godella, 64","San Cristóbal, Madrid",64.0,60.0,2,1.0,NaN,NaN,NaN,NaN,"Calle de Godella, 64",False,Calle de Godella,64,NaN,3,False,NaN,Neighborhood 135: San Cristóbal (1308.89 €/m2)...,sale,471,NaN,False,85000,1328,True,HouseType 1: Pisos,False,False,1960.0,NaN,NaN,NaN,True,NaN,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,False,NaN,NaN,NaN,NaN,False,True,False,False
1,1,21741,Piso en venta en calle de la del Manojo de Rosas,"Los Ángeles, Madrid",70.0,NaN,3,1.0,NaN,NaN,NaN,NaN,Calle de la del Manojo de Rosas,True,Calle de la del Manojo de Rosas,NaN,NaN,4,False,NaN,Neighborhood 132: Los Ángeles (1796.68 €/m2) -...,sale,666,NaN,False,129900,1856,True,HouseType 1: Pisos,True,False,NaN,NaN,NaN,NaN,NaN,True,True,True,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,en trámite,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
print(df['id'].is_unique)

True




<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <p>La colonne 'id' (index 1) ne contient que des valeurs unique, elle peut servir d'
La colonne index 0 n'apporte rien, on peut la supprimer</p>

</div>

In [94]:
df = pd.read_csv("raw_data/houses_Madrid.csv", index_col=1)
df = df.drop(df.columns[0], axis=1)
df.head(1)

,title,subtitle,sq_mt_built,sq_mt_useful,n_rooms,n_bathrooms,n_floors,sq_mt_allotment,latitude,longitude,raw_address,is_exact_address_hidden,street_name,street_number,portal,floor,is_floor_under,door,neighborhood_id,operation,rent_price,rent_price_by_area,is_rent_price_known,buy_price,buy_price_by_area,is_buy_price_known,house_type_id,is_renewal_needed,is_new_development,built_year,has_central_heating,has_individual_heating,are_pets_allowed,has_ac,has_fitted_wardrobes,has_lift,is_exterior,has_garden,has_pool,has_terrace,has_balcony,has_storage_room,is_furnished,is_kitchen_equipped,is_accessible,has_green_zones,energy_certificate,has_parking,has_private_parking,has_public_parking,is_parking_included_in_price,parking_price,is_orientation_north,is_orientation_west,is_orientation_south,is_orientation_east
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
21742,"Piso en venta en calle de Godella, 64","San Cristóbal, Madrid",64.0,60.0,2,1.0,NaN,NaN,NaN,NaN,"Calle de Godella, 64",False,Calle de Godella,64,NaN,3,False,NaN,Neighborhood 135: San Cristóbal (1308.89 €/m2)...,sale,471,NaN,False,85000,1328,True,HouseType 1: Pisos,False,False,1960.0,NaN,NaN,NaN,True,NaN,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,False,NaN,NaN,NaN,NaN,False,True,False,False


<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <p>Il y a plusieurs colonnes vide :</p>
</div>


In [95]:
empty = df.columns[df.isna().all()].tolist()
empty

['latitude',
 'longitude',
 'portal',
 'door',
 'rent_price_by_area',
 'are_pets_allowed',
 'is_furnished',
 'is_kitchen_equipped',
 'has_private_parking',
 'has_public_parking']

<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <p>Suppression des colonnes vide :</p>
</div>


In [96]:
df_clean = df.dropna(axis=1, how='all')
df_clean.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 21742 entries, 21742 to 1
Data columns (total 46 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   title                         21742 non-null  object 
 1   subtitle                      21742 non-null  object 
 2   sq_mt_built                   21616 non-null  float64
 3   sq_mt_useful                  8228 non-null   float64
 4   n_rooms                       21742 non-null  int64  
 5   n_bathrooms                   21726 non-null  float64
 6   n_floors                      1437 non-null   float64
 7   sq_mt_allotment               1432 non-null   float64
 8   raw_address                   16277 non-null  object 
 9   is_exact_address_hidden       21742 non-null  bool   
 10  street_name                   15837 non-null  object 
 11  street_number                 6300 non-null   object 
 12  floor                         19135 non-null  object 
 13  is_flo

In [97]:
df = df_clean
df.head()

,title,subtitle,sq_mt_built,sq_mt_useful,n_rooms,n_bathrooms,n_floors,sq_mt_allotment,raw_address,is_exact_address_hidden,street_name,street_number,floor,is_floor_under,neighborhood_id,operation,rent_price,is_rent_price_known,buy_price,buy_price_by_area,is_buy_price_known,house_type_id,is_renewal_needed,is_new_development,built_year,has_central_heating,has_individual_heating,has_ac,has_fitted_wardrobes,has_lift,is_exterior,has_garden,has_pool,has_terrace,has_balcony,has_storage_room,is_accessible,has_green_zones,energy_certificate,has_parking,is_parking_included_in_price,parking_price,is_orientation_north,is_orientation_west,is_orientation_south,is_orientation_east
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
21742,"Piso en venta en calle de Godella, 64","San Cristóbal, Madrid",64.0,60.0,2,1.0,NaN,NaN,"Calle de Godella, 64",False,Calle de Godella,64,3,False,Neighborhood 135: San Cristóbal (1308.89 €/m2)...,sale,471,False,85000,1328,True,HouseType 1: Pisos,False,False,1960.0,NaN,NaN,True,NaN,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,False,NaN,NaN,False,True,False,False
21741,Piso en venta en calle de la del Manojo de Rosas,"Los Ángeles, Madrid",70.0,NaN,3,1.0,NaN,NaN,Calle de la del Manojo de Rosas,True,Calle de la del Manojo de Rosas,NaN,4,False,Neighborhood 132: Los Ángeles (1796.68 €/m2) -...,sale,666,False,129900,1856,True,HouseType 1: Pisos,True,False,NaN,NaN,NaN,NaN,True,True,True,NaN,NaN,True,NaN,NaN,NaN,NaN,en trámite,False,NaN,NaN,NaN,NaN,NaN,NaN
21740,"Piso en venta en calle del Talco, 68","San Andrés, Madrid",94.0,54.0,2,2.0,NaN,NaN,"Calle del Talco, 68",False,Calle del Talco,68,1,False,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,sale,722,False,144247,1535,True,HouseType 1: Pisos,False,False,NaN,False,True,NaN,True,True,True,NaN,NaN,NaN,NaN,True,NaN,NaN,no indicado,False,NaN,NaN,NaN,NaN,NaN,NaN
21739,Piso en venta en calle Pedro Jiménez,"San Andrés, Madrid",64.0,NaN,2,1.0,NaN,NaN,Calle Pedro Jiménez,True,Calle Pedro Jiménez,NaN,Bajo,True,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,sale,583,False,109900,1717,True,HouseType 1: Pisos,False,False,1955.0,NaN,NaN,NaN,NaN,True,True,NaN,NaN,NaN,NaN,True,True,NaN,en trámite,False,NaN,NaN,False,False,True,False
21738,Piso en venta en carretera de Villaverde a Val...,"Los Rosales, Madrid",108.0,90.0,2,2.0,NaN,NaN,Carretera de Villaverde a Vallecas,True,Carretera de Villaverde a Vallecas,NaN,4,False,Neighborhood 133: Los Rosales (1827.79 €/m2) -...,sale,1094,False,260000,2407,True,HouseType 1: Pisos,False,False,2003.0,NaN,NaN,True,True,True,True,NaN,True,NaN,NaN,True,NaN,True,en trámite,True,True,0.0,True,True,True,True


<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <p>Typage des colonne de float et Nan:</p>
</div>


In [98]:
df['n_floors'].nunique()

6

In [99]:
df['n_floors'].unique()

array([nan,  2.,  3.,  4.,  1.,  5.,  7.])

In [100]:
df['n_floors'].dtype

dtype('float64')

In [101]:
# Converti en entier nullable : de 0 à 254 et NaN ou plus
df['n_floors'] = df['n_floors'].round().astype('Int8')
df['n_bathrooms'] = df['n_bathrooms'].round().astype('UInt8')
df['built_year'] = df['built_year'].round().astype('UInt16')

df['sq_mt_built'] = df['sq_mt_built'].round().astype('UInt16')
df['sq_mt_useful'] = df['sq_mt_useful'].round().astype('UInt16')

df['parking_price'] = df['parking_price'].round().astype('UInt64')
df['sq_mt_allotment'] = df['sq_mt_allotment'].round().astype('UInt64')




<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h3>Variables binaires</h3>
</div>

In [ ]:
# Détection des variables qui n'ot qu'une seule modalité
# NaN étant considéré comme une modalité .....  (dropna=False)
binary_cols = df.columns[df.nunique(dropna=False)==1].tolist()

In [ ]:
binary_cols = ['has_pool', 'has_terrace', 'has_garden', 'has_balcony', 'has_storage_room', 'has_ac', 'has_fitted_wardrobes']

for col in binary_cols:
    # convertit d'abord en booléen nullable, puis remplace NaN par False, puis UInt8
    df[col] = df[col].astype('boolean').fillna(False).astype('UInt8')


<div style="
    background-color: #439cc8; 
    color: white; 
    font-size: 16px; 
    font-style: italic; 
    padding: 10px 15px; 
    margin-bottom: 15px; 
    border-radius: 8px;">
    <h2>Export</h2>
</div>

In [114]:
df.to_feather(r'data/madrid_houses.feather')

In [115]:
df.head()

,title,subtitle,sq_mt_built,sq_mt_useful,n_rooms,n_bathrooms,n_floors,sq_mt_allotment,raw_address,is_exact_address_hidden,street_name,street_number,floor,is_floor_under,neighborhood_id,operation,rent_price,is_rent_price_known,buy_price,buy_price_by_area,is_buy_price_known,house_type_id,is_renewal_needed,is_new_development,built_year,has_central_heating,has_individual_heating,has_ac,has_fitted_wardrobes,has_lift,is_exterior,has_garden,has_pool,has_terrace,has_balcony,has_storage_room,is_accessible,has_green_zones,energy_certificate,has_parking,is_parking_included_in_price,parking_price,is_orientation_north,is_orientation_west,is_orientation_south,is_orientation_east
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
21742,"Piso en venta en calle de Godella, 64","San Cristóbal, Madrid",64,60,2,1,<NA>,<NA>,"Calle de Godella, 64",False,Calle de Godella,64,3,False,Neighborhood 135: San Cristóbal (1308.89 €/m2)...,sale,471,False,85000,1328,True,HouseType 1: Pisos,False,False,1960,NaN,NaN,1,0,False,True,0,0,0,0,0,NaN,NaN,D,False,NaN,<NA>,False,True,False,False
21741,Piso en venta en calle de la del Manojo de Rosas,"Los Ángeles, Madrid",70,<NA>,3,1,<NA>,<NA>,Calle de la del Manojo de Rosas,True,Calle de la del Manojo de Rosas,NaN,4,False,Neighborhood 132: Los Ángeles (1796.68 €/m2) -...,sale,666,False,129900,1856,True,HouseType 1: Pisos,True,False,<NA>,NaN,NaN,0,1,True,True,0,0,1,0,0,NaN,NaN,en trámite,False,NaN,<NA>,NaN,NaN,NaN,NaN
21740,"Piso en venta en calle del Talco, 68","San Andrés, Madrid",94,54,2,2,<NA>,<NA>,"Calle del Talco, 68",False,Calle del Talco,68,1,False,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,sale,722,False,144247,1535,True,HouseType 1: Pisos,False,False,<NA>,False,True,0,1,True,True,0,0,0,0,1,NaN,NaN,no indicado,False,NaN,<NA>,NaN,NaN,NaN,NaN
21739,Piso en venta en calle Pedro Jiménez,"San Andrés, Madrid",64,<NA>,2,1,<NA>,<NA>,Calle Pedro Jiménez,True,Calle Pedro Jiménez,NaN,Bajo,True,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,sale,583,False,109900,1717,True,HouseType 1: Pisos,False,False,1955,NaN,NaN,0,0,True,True,0,0,0,0,1,True,NaN,en trámite,False,NaN,<NA>,False,False,True,False
21738,Piso en venta en carretera de Villaverde a Val...,"Los Rosales, Madrid",108,90,2,2,<NA>,<NA>,Carretera de Villaverde a Vallecas,True,Carretera de Villaverde a Vallecas,NaN,4,False,Neighborhood 133: Los Rosales (1827.79 €/m2) -...,sale,1094,False,260000,2407,True,HouseType 1: Pisos,False,False,2003,NaN,NaN,1,1,True,True,0,1,0,0,1,NaN,True,en trámite,True,True,0,True,True,True,True
